<a href="https://colab.research.google.com/github/quangcrazymen/CS114.M11/blob/main/FinalProject/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Install dependency


In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## 2.Keypoints using MP Holistic

In [7]:
cap = cv2.VideoCapture(0)
while cap.isOpened():

  # Read feed
  ret, frame = cap.read()
  cv2.imshow('OpenCV Feed',frame)
  if cs2.waitKey(10) & 0xFF ==ord('q'):
    break
cap.release()
cv2.destroyAllWindows()